In [8]:
import requests
import json
from pprint import pp
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os
from tqdm import tqdm
import pandas as pd

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')

In [9]:
# 定義模型
def get_embeddings(text, dimensions = 256, model="text-embedding-3-small"):
  url = "https://api.openai.com/v1/embeddings"
  payload = { 
    "input": text, 
    "model": model, 
    "dimensions": dimensions 
    }
  headers = { 
    "Authorization": f'Bearer {openai_api_key}', 
    "Content-Type": "application/json" 
    }
  response = requests.post(url, headers=headers, data=json.dumps(payload))
  
  if response.status_code == 200 :
    return json.loads(response.text)["data"][0]["embedding"]
  else :
    print(f"錯誤（status {response.status_code}）: {response.text}")
    return None

In [10]:
# 載入 CSV
df = pd.read_csv(r"..\prompt提取技能\104_skills_新北市_1140721.csv")
df = df .drop(columns="工作技能")
df = df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 47 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   job_id  5 non-null      object 
 1   更新日期    5 non-null      object 
 2   查詢職類    5 non-null      int64  
 3   查詢關鍵字   5 non-null      object 
 4   職務類別    5 non-null      object 
 5   職缺名稱    5 non-null      object 
 6   公司名稱    5 non-null      object 
 7   公司連結    5 non-null      object 
 8   產業別     5 non-null      object 
 9   上班地點    5 non-null      object 
 10  地區      5 non-null      object 
 11  城市      5 non-null      object 
 12  國家      5 non-null      object 
 13  薪資      5 non-null      object 
 14  薪資下限    5 non-null      int64  
 15  薪資上限    5 non-null      int64  
 16  職缺描述    5 non-null      object 
 17  職務需求    0 non-null      float64
 18  工作經歷要求  5 non-null      object 
 19  學歷要求    5 non-null      object 
 20  科系要求    2 non-null      object 
 21  擅長工具    1 non-null      object 
 22  中文聽力  

In [11]:
skill_cols = [col for col in df.columns if "技能" in col and "向量" not in col]

for col in tqdm(skill_cols):
    print(f"▶️ 處理欄位：{col}")
    vec_col = f"{col}向量"  # 產生對應向量欄位名稱
    df[vec_col] = df[col].fillna("").apply(lambda x: get_embeddings(x.strip()) if x.strip() else None)

  0%|          | 0/2 [00:00<?, ?it/s]

▶️ 處理欄位：技能原文


 50%|█████     | 1/2 [00:04<00:04,  4.55s/it]

▶️ 處理欄位：技能


100%|██████████| 2/2 [00:07<00:00,  3.65s/it]


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 49 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   job_id  5 non-null      object 
 1   更新日期    5 non-null      object 
 2   查詢職類    5 non-null      int64  
 3   查詢關鍵字   5 non-null      object 
 4   職務類別    5 non-null      object 
 5   職缺名稱    5 non-null      object 
 6   公司名稱    5 non-null      object 
 7   公司連結    5 non-null      object 
 8   產業別     5 non-null      object 
 9   上班地點    5 non-null      object 
 10  地區      5 non-null      object 
 11  城市      5 non-null      object 
 12  國家      5 non-null      object 
 13  薪資      5 non-null      object 
 14  薪資下限    5 non-null      int64  
 15  薪資上限    5 non-null      int64  
 16  職缺描述    5 non-null      object 
 17  職務需求    0 non-null      float64
 18  工作經歷要求  5 non-null      object 
 19  學歷要求    5 non-null      object 
 20  科系要求    2 non-null      object 
 21  擅長工具    1 non-null      object 
 22  中文聽力  

In [6]:

skill_vector_cols = [col for col in df.columns if col.startswith("技能") and col.endswith("向量")]

# 對每門課程組成清單
def collect_course_skill_vectors(row):
    skill_vectors = []
    for vec_col in skill_vector_cols:
        skill_num = vec_col.replace("向量", "")
        skill_name = row.get(skill_num)
        vec = row.get(vec_col)
        if isinstance(vec, list) and skill_name:  # 確保是有效向量
            skill_vectors.append({"技能名": skill_name, "向量": vec})
    return skill_vectors

df["職缺技能向量清單"] = df.apply(collect_course_skill_vectors, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 50 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   job_id    5 non-null      object 
 1   更新日期      5 non-null      object 
 2   查詢職類      5 non-null      int64  
 3   查詢關鍵字     5 non-null      object 
 4   職務類別      5 non-null      object 
 5   職缺名稱      5 non-null      object 
 6   公司名稱      5 non-null      object 
 7   公司連結      5 non-null      object 
 8   產業別       5 non-null      object 
 9   上班地點      5 non-null      object 
 10  地區        5 non-null      object 
 11  城市        5 non-null      object 
 12  國家        5 non-null      object 
 13  薪資        5 non-null      object 
 14  薪資下限      5 non-null      int64  
 15  薪資上限      5 non-null      int64  
 16  職缺描述      5 non-null      object 
 17  職務需求      0 non-null      float64
 18  工作經歷要求    5 non-null      object 
 19  學歷要求      5 non-null      object 
 20  科系要求      2 non-null      object 
 2

In [7]:
df.to_csv("104_skills_embedding_1140721.csv", index=False, encoding="utf-8-sig")